In [ ]:
# Google Colab 환경을 위한 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import Libraries
import pandas as pd
import glob
import os

# 1. Data Loading
# Google Colab 경로에 맞게 train.csv 파일 로드
train_df = pd.read_csv('/content/drive/MyDrive/lg_aimers_2/data/train/train.csv')


# 2. 전처리 함수 정의
def preprocess_data(df):
    """
    주어진 DataFrame에 전처리 및 피처 엔지니어링을 수행하는 함수
    """
    # '영업일자' 컬럼을 datetime 객체로 변환
    df['영업일자'] = pd.to_datetime(df['영업일자'])

    # 날짜 관련 피처 생성
    df['년'] = df['영업일자'].dt.year
    df['월'] = df['영업일자'].dt.month
    df['일'] = df['영업일자'].dt.day
    df['요일'] = df['영업일자'].dt.dayofweek  # 월요일=0, 일요일=6
    df['주말여부'] = df['요일'].isin([5, 6])

    # 공휴일 정보 추가
    holidays_2023 = ['2023-01-01', '2023-01-21', '2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01', '2023-05-05', '2023-05-27', '2023-05-29', '2023-06-06', '2023-08-15', '2023-09-28', '2023-09-29', '2023-09-30', '2023-10-03', '2023-10-09', '2023-12-25']
    holidays_2024 = ['2024-01-01', '2024-02-09', '2024-02-10', '2024-02-11', '2024-02-12', '2024-03-01', '2024-04-10', '2024-05-05', '2024-05-06', '2024-05-15', '2024-06-06', '2024-08-15', '2024-09-16', '2024-09-17', '2024-09-18', '2024-10-03', '2024-10-09', '2024-12-25']
    holidays_2025 = ['2025-01-01', '2025-01-28', '2025-01-29', '2025-01-30', '2025-03-01', '2025-03-03', '2025-05-05', '2025-05-06', '2025-06-06', '2025-08-15', '2025-10-03', '2025-10-05', '2025-10-06', '2025-10-07', '2025-10-08', '2025-10-09', '2025-12-25']

    all_holidays = holidays_2023 + holidays_2024 + holidays_2025
    all_holidays_df = pd.DataFrame(pd.to_datetime(all_holidays), columns=['영업일자'])
    all_holidays_df['공휴일여부'] = True

    df = pd.merge(df, all_holidays_df, on='영업일자', how='left')
    df['공휴일여부'] = df['공휴일여부'].fillna(False)

    # 신규 메뉴 출시 정보 추가
    launch_dates = {
        '느티나무 셀프BBQ_1인 수저세트': '2023-01-17', '느티나무 셀프BBQ_BBQ55(단체)': '2023-01-05',
        '느티나무 셀프BBQ_대여료 90,000원': '2023-01-02', '느티나무 셀프BBQ_본삼겹 (단품,실내)': '2023-01-03',
        '느티나무 셀프BBQ_스프라이트 (단체)': '2023-01-03', '느티나무 셀프BBQ_신라면': '2023-04-14',
        '느티나무 셀프BBQ_쌈야채세트': '2023-01-11', '느티나무 셀프BBQ_쌈장': '2023-04-14',
        '느티나무 셀프BBQ_육개장 사발면': '2023-04-14', '느티나무 셀프BBQ_일회용 소주컵': '2023-01-23',
        '느티나무 셀프BBQ_일회용 종이컵': '2023-01-22', '느티나무 셀프BBQ_잔디그늘집 대여료 (12인석)': '2023-04-14',
        '느티나무 셀프BBQ_잔디그늘집 대여료 (6인석)': '2023-01-05', '느티나무 셀프BBQ_잔디그늘집 의자 추가': '2023-04-14',
        '느티나무 셀프BBQ_참이슬 (단체)': '2023-01-03', '느티나무 셀프BBQ_친환경 접시 14cm': '2023-01-22',
        '느티나무 셀프BBQ_친환경 접시 23cm': '2023-01-05', '느티나무 셀프BBQ_카스 병(단체)': '2023-01-03',
        '느티나무 셀프BBQ_콜라 (단체)': '2023-01-03', '느티나무 셀프BBQ_햇반': '2023-04-14',
        '느티나무 셀프BBQ_허브솔트': '2023-04-14', '담하_(단체) 공깃밥': '2023-03-13',
        '담하_(단체) 생목살 김치전골 2.0': '2023-09-18', '담하_(단체) 은이버섯 갈비탕': '2023-06-12',
        '담하_(단체) 한우 우거지 국밥': '2023-01-06', '담하_(단체) 황태해장국 3/27까지': '2023-01-07',
        '담하_(정식) 된장찌개': '2023-06-03', '담하_(정식) 물냉면 ': '2023-06-03',
        '담하_(정식) 비빔냉면': '2023-06-03', '담하_(후식) 물냉면': '2023-06-02',
        '담하_(후식) 비빔냉면': '2023-06-02', '담하_갑오징어 비빔밥': '2023-03-17',
        '담하_갱시기': '2023-12-08', '담하_꼬막 비빔밥': '2023-09-08',
        '담하_느린마을 막걸리': '2023-01-02', '담하_담하 한우 불고기 정식': '2023-06-02',
        '담하_더덕 한우 지짐': '2023-09-09', '담하_라면사리': '2023-01-04',
        '담하_룸 이용료': '2023-01-03', '담하_명인안동소주': '2023-07-01',
        '담하_명태회 비빔냉면': '2023-06-02', '담하_문막 복분자 칵테일': '2023-09-12',
        '담하_봉평메밀 물냉면': '2023-06-02', '담하_제로콜라': '2023-01-05',
        '담하_처음처럼': '2023-01-03', '담하_하동 매실 칵테일': '2023-03-18',
        '라그로타_AUS (200g)': '2023-12-08', '라그로타_G-Charge(3)': '2023-01-02',
        '라그로타_Open Food': '2023-01-07', '라그로타_그릴드 비프 샐러드': '2023-09-08',
        '라그로타_까르보나라': '2023-12-08', '라그로타_모둠 해산물 플래터': '2023-09-09',
        '라그로타_미션 서드 카베르네 쉬라': '2023-01-02', '라그로타_버섯 크림 리조또': '2023-12-08',
        '라그로타_시저 샐러드 ': '2023-09-08', '라그로타_알리오 에 올리오 ': '2023-09-08',
        '라그로타_양갈비 (4ps)': '2023-09-10', '라그로타_한우 (200g)': '2023-12-09',
        '라그로타_해산물 토마토 스튜 파스타': '2023-12-08', '미라시아_(단체)브런치주중 36,000': '2023-01-03',
        '미라시아_(오븐) 하와이안 쉬림프 피자': '2023-09-09', '미라시아_BBQ 고기추가': '2023-01-05',
        '미라시아_글라스와인 (레드)': '2023-01-02', '미라시아_레인보우칵테일(알코올)': '2023-01-02',
        '미라시아_버드와이저(무제한)': '2023-04-21', '미라시아_보일링 랍스타 플래터': '2023-06-05',
        '미라시아_보일링 랍스타 플래터(덜매운맛)': '2023-06-03', '미라시아_브런치(대인) 주중': '2023-01-02',
        '미라시아_쉬림프 투움바 파스타': '2023-06-03', '미라시아_스텔라(무제한)': '2023-04-21',
        '미라시아_스프라이트': '2023-06-02', '미라시아_얼그레이 하이볼': '2023-01-02',
        '미라시아_유자 하이볼': '2023-03-17', '미라시아_잭 애플 토닉': '2023-09-09',
        '미라시아_칠리 치즈 프라이': '2023-06-03', '미라시아_코카콜라': '2023-06-02',
        '미라시아_코카콜라(제로)': '2023-06-12', '미라시아_콥 샐러드': '2023-12-08',
        '미라시아_파스타면 추가(150g)': '2023-06-03', '미라시아_핑크레몬에이드': '2023-03-17',
        '연회장_Cass Beer': '2023-01-06', '연회장_Conference L1': '2023-01-03',
        '연회장_Conference L2': '2023-01-11', '연회장_Conference L3': '2023-01-05',
        '연회장_Conference M1': '2023-01-06', '연회장_Conference M8': '2023-01-09',
        '연회장_Conference M9': '2023-01-06', '연회장_Convention Hall': '2023-01-03',
        '연회장_Cookie Platter': '2023-01-09', '연회장_Grand Ballroom': '2023-01-06',
        '연회장_OPUS 2': '2023-01-05', '연회장_Regular Coffee': '2023-02-24',
        '연회장_공깃밥': '2023-07-21', '연회장_마라샹궈': '2023-09-08',
        '연회장_매콤 무뼈닭발&계란찜': '2023-01-02', '연회장_삼겹살추가 (200g)': '2023-07-21',
        '연회장_왕갈비치킨': '2023-07-22', '카페테리아_단체식 13000(신)': '2023-04-18',
        '카페테리아_단체식 18000(신)': '2023-04-05', '카페테리아_진사골 설렁탕': '2023-12-06',
        '카페테리아_한상 삼겹구이 정식(2인) 소요시간 약 15~20분': '2023-03-17',
        '화담숲주막_느린마을 막걸리': '2023-03-31', '화담숲주막_단호박 식혜 ': '2023-03-31',
        '화담숲주막_병천순대': '2023-03-31', '화담숲주막_스프라이트': '2023-03-31',
        '화담숲주막_참살이 막걸리': '2023-03-31', '화담숲주막_찹쌀식혜': '2023-03-31',
        '화담숲주막_콜라': '2023-03-31', '화담숲주막_해물파전': '2023-03-31',
        '화담숲카페_메밀미숫가루': '2023-03-31', '화담숲카페_아메리카노 HOT': '2023-03-31',
        '화담숲카페_아메리카노 ICE': '2023-03-31', '화담숲카페_카페라떼 ICE': '2023-03-31',
        '화담숲카페_현미뻥스크림': '2023-03-31'
    }
    launch_dates = {k: pd.to_datetime(v) for k, v in launch_dates.items()}

    def calculate_days_since_launch(row):
        menu = row['영업장명_메뉴명']
        if menu in launch_dates:
            launch_date = launch_dates[menu]
            if row['영업일자'] >= launch_date:
                return (row['영업일자'] - launch_date).days
        return 0

    df['출시일로부터경과일'] = df.apply(calculate_days_since_launch, axis=1)

    # '신규메뉴여부' 피처 생성
    df['신규메뉴여부'] = df['영업장명_메뉴명'].isin(launch_dates.keys())

    # '영업장명_메뉴명' 컬럼 분리
    df[['영업장명', '메뉴명']] = df['영업장명_메뉴명'].str.split('_', n=1, expand=True)

    return df

# 3. 각 데이터셋에 전처리 함수 적용
train_preprocessed = preprocess_data(train_df.copy())

# 4. 전처리된 데이터 저장
train_preprocessed_path = '/content/drive/MyDrive/lg_aimers_2/train_preprocessed.csv'

train_preprocessed.to_csv(train_preprocessed_path, index=False, encoding='utf-8-sig')

print("데이터 전처리 과정이 모두 완료되었습니다.")
print(f"전처리된 학습 데이터가 '{train_preprocessed_path}' 파일로 저장되었습니다.")

/tmp/ipython-input-2572583762.py:36: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['공휴일여부'] = df['공휴일여부'].fillna(False)


데이터 전처리 과정이 모두 완료되었습니다.
전처리된 학습 데이터가 '/content/drive/MyDrive/lg_aimers_2/train_preprocessed.csv' 파일로 저장되었습니다.


In [ ]:
# !pip install lightgbm -q

import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor, log_evaluation, early_stopping
from sklearn.metrics import mean_squared_error
import joblib, os

TRAIN_PATH = "/content/drive/MyDrive/lg_aimers_2/train_preprocessed.csv"

# 1) Load
df = pd.read_csv(TRAIN_PATH, parse_dates=["영업일자"])

# 2) 기본 타입 정리
bool_cols = [c for c in ["주말여부","공휴일여부","신규메뉴여부"] if c in df.columns]
for c in bool_cols:
    df[c] = df[c].astype(int)

# 원-소스 카테고리: 영업장명_메뉴명 (분리된 '영업장명','메뉴명'이 있어도 식별키는 이것 하나면 충분)
key_col = "영업장명_메뉴명"
if key_col not in df.columns:
    raise ValueError(f"'{key_col}' 컬럼이 필요합니다.")

df[key_col] = df[key_col].astype("category")

# 3) 누수 방지용 시계열 정렬 후 lag/rolling 생성
df = df.sort_values([key_col, "영업일자"]).copy()

lag_list = [1, 7, 14, 28]
for lag in lag_list:
    df[f"lag_{lag}"] = df.groupby(key_col)["매출수량"].shift(lag)

# 과거만 사용하는 rolling: shift(1) 한 뒤 rolling
def add_rolling(g, window, how="mean"):
    s = g["매출수량"].shift(1)  # 반드시 과거만
    if how == "mean":
        return s.rolling(window, min_periods=1).mean()
    elif how == "sum":
        return s.rolling(window, min_periods=1).sum()
    else:
        raise ValueError("how must be 'mean' or 'sum'")

df["roll7_mean"]  = df.groupby(key_col, group_keys=False).apply(add_rolling, window=7, how="mean")
df["roll7_sum"]   = df.groupby(key_col, group_keys=False).apply(add_rolling, window=7, how="sum")
df["roll14_mean"] = df.groupby(key_col, group_keys=False).apply(add_rolling, window=14, how="mean")
df["roll28_mean"] = df.groupby(key_col, group_keys=False).apply(add_rolling, window=28, how="mean")

# 같은 요일 패턴(최근 4주 내 같은 요일 평균)
def same_dow_mean_28(g):
    out = np.full(len(g), np.nan, dtype=float)
    vals = g["매출수량"].shift(1)
    dows = g["요일"]
    for i in range(len(g)):
        lo = max(0, i-28)
        hist = range(lo, i)
        same = [j for j in hist if dows.iloc[j] == dows.iloc[i]]
        if same:
            out[i] = vals.iloc[same].mean()
    return pd.Series(out, index=g.index)

df["same_dow_mean_28"] = df.groupby(key_col, group_keys=False).apply(same_dow_mean_28)

# 신규 메뉴 활용 추가 파생(선택적): 출시 후 경과 주차 구간
if "출시일로부터경과일" in df.columns:
    df["출시후_주차"] = (df["출시일로부터경과일"] // 7).clip(lower=0).astype(int)
    # 초반 효과 캡처용 구간 더미(0주/1~2주/3~4주/5주+)
    df["출시_0주"]   = (df["출시후_주차"] == 0).astype(int)
    df["출시_1_2주"] = df["출시후_주차"].between(1,2).astype(int)
    df["출시_3_4주"] = df["출시후_주차"].between(3,4).astype(int)
    df["출시_5주이상"] = (df["출시후_주차"] >= 5).astype(int)

# 4) 학습/검증 분리 (시간 순)
train_cut   = pd.Timestamp("2024-05-31")
valid_start = pd.Timestamp("2024-06-01")
valid_end   = pd.Timestamp("2024-06-15")

train_df = df[df["영업일자"] <= train_cut].copy()
valid_df = df[(df["영업일자"] >= valid_start) & (df["영업일자"] <= valid_end)].copy()

# 사용 피처 정의
base_feats = [c for c in ["년","월","일","요일","주말여부","공휴일여부","신규메뉴여부","출시일로부터경과일",
                          "출시후_주차","출시_0주","출시_1_2주","출시_3_4주","출시_5주이상"] if c in df.columns]
lag_feats  = [f"lag_{l}" for l in lag_list]
roll_feats = [c for c in ["roll7_mean","roll7_sum","roll14_mean","roll28_mean","same_dow_mean_28"] if c in df.columns]

use_cols = base_feats + lag_feats + roll_feats + [key_col]

# lag가 없는 초기구간 제거(필수)
train_df = train_df.dropna(subset=[f"lag_{min(lag_list)}"])
valid_df = valid_df.dropna(subset=[f"lag_{min(lag_list)}"])

# 남은 결측 대체
fill_cols = list(set(use_cols) - {key_col})
train_df[fill_cols] = train_df[fill_cols].fillna(0)
valid_df[fill_cols] = valid_df[fill_cols].fillna(0)

X_trn = train_df[use_cols].copy()
y_trn = train_df["매출수량"].astype(float).values
X_val = valid_df[use_cols].copy()
y_val = valid_df["매출수량"].astype(float).values

# LightGBM 범주형 지정 (pandas category로 캐스팅되어 있어야 함)
cat_features = [use_cols.index(key_col)]

# 5) 모델 선언 & 학습 (버전 호환: callbacks로 로그/조기종료)
lgbm = LGBMRegressor(
    objective="regression",
    n_estimators=3000,
    learning_rate=0.03,
    max_depth=-1,
    num_leaves=127,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1
)

lgbm.fit(
    X_trn, y_trn,
    eval_set=[(X_val, y_val)],
    eval_metric="rmse",
    categorical_feature=cat_features,
    callbacks=[
        log_evaluation(period=200),
        early_stopping(stopping_rounds=200)
    ],
)

# 6) 검증 성능 (버전 호환: RMSE = sqrt(MSE))
val_pred = lgbm.predict(X_val, num_iteration=lgbm.best_iteration_)
mse = mean_squared_error(y_val, val_pred)
rmse = np.sqrt(mse)
print(f"[VALID] RMSE: {rmse:,.4f}")

# 7) 피처 중요도 & 아티팩트 저장
imp = pd.DataFrame({
    "feature": use_cols,
    "importance": lgbm.feature_importances_
}).sort_values("importance", ascending=False)

os.makedirs("/content/models", exist_ok=True)
joblib.dump(
    {"model": lgbm, "features": use_cols, "cat_idx": cat_features},
    "/content/drive/MyDrive/lg_aimers_2/models/lgbm_menu_demand.pkl"
)
imp.to_csv("/content/drive/MyDrive/lg_aimers_2/models/feature_importance.csv", index=False)

print("모델 저장: /content/drive/MyDrive/lg_aimers_2/models/lgbm_menu_demand.pkl")
print("중요도 저장: /content/drive/MyDrive/lg_aimers_2/models/feature_importance.csv")
print("상위 중요도:\n", imp.head(15))


/tmp/ipython-input-3642834567.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f"lag_{lag}"] = df.groupby(key_col)["매출수량"].shift(lag)
/tmp/ipython-input-3642834567.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f"lag_{lag}"] = df.groupby(key_col)["매출수량"].shift(lag)
/tmp/ipython-input-3642834567.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f"lag_{lag}"] = df.groupby(key_col)["매출수량"].shift(lag)
/

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.060741 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2886
[LightGBM] [Info] Number of data points in the train set: 99588, number of used features: 23
[LightGBM] [Info] Start training from score 10.723581
Training until validation scores don't improve for 200 rounds
[200]	valid_0's rmse: 15.5386	valid_0's l2: 241.447
[400]	valid_0's rmse: 15.3516	valid_0's l2: 235.671
[600]	valid_0's rmse: 15.2393	valid_0's l2: 232.236
[800]	valid_0's rmse: 15.2412	valid_0's l2: 232.293
Early stopping, best iteration is:
[665]	valid_0's rmse: 15.2167	valid_0's l2: 231.548
[VALID] RMSE: 15.2167
모델 저장: /content/drive/MyDrive/lg_aimers_2/models/lgbm_menu_demand.pkl
중요도 저장: /content/drive/MyDrive/lg_aimers_2/models/feature_importance.csv
상위 중요도:
              feature  importance
13             lag_1        9

In [ ]:
# !pip install lightgbm -q
import os, glob, joblib
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor

# ===== 경로 설정 =====
MODEL_PATH   = "/content/drive/MyDrive/lg_aimers_2/models/lgbm_menu_demand.pkl"
TEST_DIR     = "/content/drive/MyDrive/lg_aimers_2/data/test"   # TEST_00.csv ~ TEST_09.csv
SAMPLE_PATH  = "/content/drive/MyDrive/lg_aimers_2/data/sample_submission.csv"  # 샘플 제출 파일 경로로 바꿔줘
OUT_PATH     = "/content/drive/MyDrive/lg_aimers_2/submission_lightgbm_01.csv"

# ===== 모델 로드 =====
bundle = joblib.load(MODEL_PATH)
model: LGBMRegressor = bundle["model"]
use_cols = bundle["features"]
cat_idx  = bundle["cat_idx"]
cat_col  = use_cols[cat_idx[0]]

# ===== 달력/공휴일/출시정보 (전처리 때와 동일) =====
holidays_2023 = ['2023-01-01','2023-01-21','2023-01-22','2023-01-23','2023-01-24','2023-03-01',
                 '2023-05-05','2023-05-27','2023-05-29','2023-06-06','2023-08-15','2023-09-28',
                 '2023-09-29','2023-09-30','2023-10-03','2023-10-09','2023-12-25']
holidays_2024 = ['2024-01-01','2024-02-09','2024-02-10','2024-02-11','2024-02-12','2024-03-01',
                 '2024-04-10','2024-05-05','2024-05-06','2024-05-15','2024-06-06','2024-08-15',
                 '2024-09-16','2024-09-17','2024-09-18','2024-10-03','2024-10-09','2024-12-25']
holidays_2025 = ['2025-01-01','2025-01-28','2025-01-29','2025-01-30','2025-03-01','2025-03-03',
                 '2025-05-05','2025-05-06','2025-06-06','2025-08-15','2025-10-03','2025-10-05',
                 '2025-10-06','2025-10-07','2025-10-08','2025-10-09','2025-12-25']
HOLIDAYS = set(pd.to_datetime(holidays_2023 + holidays_2024 + holidays_2025))

launch_dates = {
    '느티나무 셀프BBQ_1인 수저세트': '2023-01-17', '느티나무 셀프BBQ_BBQ55(단체)': '2023-01-05',
    '느티나무 셀프BBQ_대여료 90,000원': '2023-01-02', '느티나무 셀프BBQ_본삼겹 (단품,실내)': '2023-01-03',
    '느티나무 셀프BBQ_스프라이트 (단체)': '2023-01-03', '느티나무 셀프BBQ_신라면': '2023-04-14',
    '느티나무 셀프BBQ_쌈야채세트': '2023-01-11', '느티나무 셀프BBQ_쌈장': '2023-04-14',
    '느티나무 셀프BBQ_육개장 사발면': '2023-04-14', '느티나무 셀프BBQ_일회용 소주컵': '2023-01-23',
    '느티나무 셀프BBQ_일회용 종이컵': '2023-01-22', '느티나무 셀프BBQ_잔디그늘집 대여료 (12인석)': '2023-04-14',
    '느티나무 셀프BBQ_잔디그늘집 대여료 (6인석)': '2023-01-05', '느티나무 셀프BBQ_잔디그늘집 의자 추가': '2023-04-14',
    '느티나무 셀프BBQ_참이슬 (단체)': '2023-01-03', '느티나무 셀프BBQ_친환경 접시 14cm': '2023-01-22',
    '느티나무 셀프BBQ_친환경 접시 23cm': '2023-01-05', '느티나무 셀프BBQ_카스 병(단체)': '2023-01-03',
    '느티나무 셀프BBQ_콜라 (단체)': '2023-01-03', '느티나무 셀프BBQ_햇반': '2023-04-14',
    '느티나무 셀프BBQ_허브솔트': '2023-04-14', '담하_(단체) 공깃밥': '2023-03-13',
    '담하_(단체) 생목살 김치전골 2.0': '2023-09-18', '담하_(단체) 은이버섯 갈비탕': '2023-06-12',
    '담하_(단체) 한우 우거지 국밥': '2023-01-06', '담하_(단체) 황태해장국 3/27까지': '2023-01-07',
    '담하_(정식) 된장찌개': '2023-06-03', '담하_(정식) 물냉면 ': '2023-06-03',
    '담하_(정식) 비빔냉면': '2023-06-03', '담하_(후식) 물냉면': '2023-06-02',
    '담하_(후식) 비빔냉면': '2023-06-02', '담하_갑오징어 비빔밥': '2023-03-17',
    '담하_갱시기': '2023-12-08', '담하_꼬막 비빔밥': '2023-09-08',
    '담하_느린마을 막걸리': '2023-01-02', '담하_담하 한우 불고기 정식': '2023-06-02',
    '담하_더덕 한우 지짐': '2023-09-09', '담하_라면사리': '2023-01-04',
    '담하_룸 이용료': '2023-01-03', '담하_명인안동소주': '2023-07-01',
    '담하_명태회 비빔냉면': '2023-06-02', '담하_문막 복분자 칵테일': '2023-09-12',
    '담하_봉평메밀 물냉면': '2023-06-02', '담하_제로콜라': '2023-01-05',
    '담하_처음처럼': '2023-01-03', '담하_하동 매실 칵테일': '2023-03-18',
    '라그로타_AUS (200g)': '2023-12-08', '라그로타_G-Charge(3)': '2023-01-02',
    '라그로타_Open Food': '2023-01-07', '라그로타_그릴드 비프 샐러드': '2023-09-08',
    '라그로타_까르보나라': '2023-12-08', '라그로타_모둠 해산물 플래터': '2023-09-09',
    '라그로타_미션 서드 카베르네 쉬라': '2023-01-02', '라그로타_버섯 크림 리조또': '2023-12-08',
    '라그로타_시저 샐러드 ': '2023-09-08', '라그로타_알리오 에 올리오 ': '2023-09-08',
    '라그로타_양갈비 (4ps)': '2023-09-10', '라그로타_한우 (200g)': '2023-12-09',
    '라그로타_해산물 토마토 스튜 파스타': '2023-12-08',
    '미라시아_(단체)브런치주중 36,000': '2023-01-03',
    '미라시아_(오븐) 하와이안 쉬림프 피자': '2023-09-09', '미라시아_BBQ 고기추가': '2023-01-05',
    '미라시아_글라스와인 (레드)': '2023-01-02', '미라시아_레인보우칵테일(알코올)': '2023-01-02',
    '미라시아_버드와이저(무제한)': '2023-04-21', '미라시아_보일링 랍스타 플래터': '2023-06-05',
    '미라시아_보일링 랍스타 플래터(덜매운맛)': '2023-06-03', '미라시아_브런치(대인) 주중': '2023-01-02',
    '미라시아_쉬림프 투움바 파스타': '2023-06-03', '미라시아_스텔라(무제한)': '2023-04-21',
    '미라시아_스프라이트': '2023-06-02', '미라시아_얼그레이 하이볼': '2023-01-02',
    '미라시아_유자 하이볼': '2023-03-17', '미라시아_잭 애플 토닉': '2023-09-09',
    '미라시아_칠리 치즈 프라이': '2023-06-03', '미라시아_코카콜라': '2023-06-02',
    '미라시아_코카콜라(제로)': '2023-06-12', '미라시아_콥 샐러드': '2023-12-08',
    '미라시아_파스타면 추가(150g)': '2023-06-03', '미라시아_핑크레몬에이드': '2023-03-17',
    '연회장_Cass Beer': '2023-01-06', '연회장_Conference L1': '2023-01-03',
    '연회장_Conference L2': '2023-01-11', '연회장_Conference L3': '2023-01-05',
    '연회장_Conference M1': '2023-01-06', '연회장_Conference M8': '2023-01-09',
    '연회장_Conference M9': '2023-01-06', '연회장_Convention Hall': '2023-01-03',
    '연회장_Cookie Platter': '2023-01-09', '연회장_Grand Ballroom': '2023-01-06',
    '연회장_OPUS 2': '2023-01-05', '연회장_Regular Coffee': '2023-02-24',
    '연회장_공깃밥': '2023-07-21', '연회장_마라샹궈': '2023-09-08',
    '연회장_매콤 무뼈닭발&계란찜': '2023-01-02', '연회장_삼겹살추가 (200g)': '2023-07-21',
    '연회장_왕갈비치킨': '2023-07-22', '카페테리아_단체식 13000(신)': '2023-04-18',
    '카페테리아_단체식 18000(신)': '2023-04-05', '카페테리아_진사골 설렁탕': '2023-12-06',
    '카페테리아_한상 삼겹구이 정식(2인) 소요시간 약 15~20분': '2023-03-17',
    '화담숲주막_느린마을 막걸리': '2023-03-31', '화담숲주막_단호박 식혜 ': '2023-03-31',
    '화담숲주막_병천순대': '2023-03-31', '화담숲주막_스프라이트': '2023-03-31',
    '화담숲주막_참살이 막걸리': '2023-03-31', '화담숲주막_찹쌀식혜': '2023-03-31',
    '화담숲주막_콜라': '2023-03-31', '화담숲주막_해물파전': '2023-03-31',
    '화담숲카페_메밀미숫가루': '2023-03-31', '화담숲카페_아메리카노 HOT': '2023-03-31',
    '화담숲카페_아메리카노 ICE': '2023-03-31', '화담숲카페_카페라떼 ICE': '2023-03-31',
    '화담숲카페_현미뻥스크림': '2023-03-31'
}
launch_dates = {k: pd.to_datetime(v) for k, v in launch_dates.items()}

def add_future_meta_row(date, key):
    row = pd.DataFrame({"영업일자":[pd.Timestamp(date)], "영업장명_메뉴명":[key]})
    row["년"] = row["영업일자"].dt.year
    row["월"] = row["영업일자"].dt.month
    row["일"] = row["영업일자"].dt.day
    row["요일"] = row["영업일자"].dt.dayofweek
    row["주말여부"] = row["요일"].isin([5,6]).astype(int)
    row["공휴일여부"] = row["영업일자"].isin(HOLIDAYS).astype(int)
    row["신규메뉴여부"] = int(key in launch_dates)
    if key in launch_dates and row.loc[0,"영업일자"] >= launch_dates[key]:
        row["출시일로부터경과일"] = int((row.loc[0,"영업일자"] - launch_dates[key]).days)
    else:
        row["출시일로부터경과일"] = 0
    # 출시 주차 더미 (학습 때 사용했다면 동일)
    row["출시후_주차"] = (row["출시일로부터경과일"] // 7)
    row["출시_0주"]   = (row["출시후_주차"] == 0).astype(int)
    row["출시_1_2주"] = row["출시후_주차"].between(1,2).astype(int)
    row["출시_3_4주"] = row["출시후_주차"].between(3,4).astype(int)
    row["출시_5주이상"] = (row["출시후_주차"] >= 5).astype(int)
    return row

def predict_group_autoreg(g: pd.DataFrame):
    key = g[cat_col].iloc[0]
    g = g.sort_values("영업일자").copy()
    g = g.dropna(subset=["매출수량"])
    assert len(g) >= 28, f"{key}: 28일 히스토리 부족"

    hist_vals = g["매출수량"].values.tolist()[-28:]
    hist_dates = g["영업일자"].tolist()[-28:]
    last_date = g["영업일자"].max()
    preds = []

    for h in range(1, 8):
        cur_date = last_date + pd.Timedelta(days=h)
        row = add_future_meta_row(cur_date, key)

        def lag(n): return hist_vals[-n] if len(hist_vals) >= n else np.nan
        row["lag_1"],row["lag_7"],row["lag_14"],row["lag_28"] = lag(1),lag(7),lag(14),lag(28)

        def rmean(n):
            arr = hist_vals[-n:] if len(hist_vals) else []
            return float(np.mean(arr)) if arr else 0.0
        def rsum(n):
            arr = hist_vals[-n:] if len(hist_vals) else []
            return float(np.sum(arr)) if arr else 0.0
        row["roll7_mean"],row["roll7_sum"],row["roll14_mean"],row["roll28_mean"] = \
            rmean(7),rsum(7),rmean(14),rmean(28)

        cur_dow = cur_date.dayofweek
        hist_dows = [pd.Timestamp(d).dayofweek for d in hist_dates]
        same_idx = [i for i in range(len(hist_vals)) if hist_dows[i]==cur_dow]
        row["same_dow_mean_28"] = float(np.mean([hist_vals[i] for i in same_idx])) if same_idx else 0.0

        X = row.reindex(columns=use_cols).copy()
        X[cat_col] = X[cat_col].astype("category")
        for c in X.columns:
            if c != cat_col:
                X[c] = X[c].fillna(0)

        yhat = float(model.predict(X, num_iteration=getattr(model, "best_iteration_", None))[0])
        yhat = max(0.0, yhat)  # 음수 방지
        preds.append(yhat)

        hist_vals.append(yhat); hist_dates.append(cur_date)
        if len(hist_vals) > 28:
            hist_vals, hist_dates = hist_vals[-28:], hist_dates[-28:]

    return preds  # 길이 7

# ===== 샘플 제출 파일 불러오기 (초기값 0) =====
sub = pd.read_csv(SAMPLE_PATH)
menu_cols = [c for c in sub.columns if c != "영업일자"]

# ===== TEST 파일 순회하며 sub 채우기 =====
test_files = sorted(glob.glob(os.path.join(TEST_DIR, "TEST_*.csv")))
print("Found:", test_files)

for f in test_files:
    test_name = os.path.splitext(os.path.basename(f))[0]  # TEST_00
    test_id = test_name.split("_")[1]                     # 00
    df = pd.read_csv(f, parse_dates=["영업일자"])
    # 타입/정렬
    if cat_col in df.columns:
        df[cat_col] = df[cat_col].astype("category")
    df = df.sort_values([cat_col,"영업일자"]).copy()

    # 메뉴 단위 예측
    for key, g in df.groupby(cat_col, observed=True):
        preds7 = predict_group_autoreg(g)
        # 제출 파일의 해당 메뉴 열이 없으면 skip (안 나오는 메뉴일 수 있음)
        if key not in menu_cols:
            continue
        # 7행( +1~+7일 ) 채우기
        for k in range(1, 8):
            ridx = sub.index[sub["영업일자"] == f"{test_name}+{k}일"]
            if len(ridx) == 1:
                sub.loc[ridx[0], key] = preds7[k-1]

# (선택) 반올림/정수화 규칙이 있다면 여기에 적용
# sub[menu_cols] = sub[menu_cols].clip(lower=0).round(4)  # 예: 소수4자리

# ===== 저장 =====
sub.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
print("Saved submission ->", OUT_PATH)


Found: ['/content/drive/MyDrive/lg_aimers_2/data/test/TEST_00.csv', '/content/drive/MyDrive/lg_aimers_2/data/test/TEST_01.csv', '/content/drive/MyDrive/lg_aimers_2/data/test/TEST_02.csv', '/content/drive/MyDrive/lg_aimers_2/data/test/TEST_03.csv', '/content/drive/MyDrive/lg_aimers_2/data/test/TEST_04.csv', '/content/drive/MyDrive/lg_aimers_2/data/test/TEST_05.csv', '/content/drive/MyDrive/lg_aimers_2/data/test/TEST_06.csv', '/content/drive/MyDrive/lg_aimers_2/data/test/TEST_07.csv', '/content/drive/MyDrive/lg_aimers_2/data/test/TEST_08.csv', '/content/drive/MyDrive/lg_aimers_2/data/test/TEST_09.csv']


/tmp/ipython-input-1543734838.py:191: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '8.791266335038811' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sub.loc[ridx[0], key] = preds7[k-1]
/tmp/ipython-input-1543734838.py:191: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.9817391245869054' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sub.loc[ridx[0], key] = preds7[k-1]
/tmp/ipython-input-1543734838.py:191: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.648284394984041' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sub.loc[ridx[0], key] = preds7[k-1]
/tmp/ipython-input-1543734838.py:191: FutureWarning: Setting an it

Saved submission -> /content/drive/MyDrive/lg_aimers_2/submission_lightgbm_01.csv
